For a start to visualizing the data and understanding the attributes we imported pandas and matplotlib

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
%matplotlib inline

we then put the given train csv into a pandas dataframe

In [3]:
df = pd.read_csv("application_train.csv")
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_seq_items",None)

In [4]:
df.loc[df['FLAG_OWN_CAR'] == 'N', 'OWN_CAR_AGE'] = 0

In [5]:
df['AMT_ANNUITY'] = df['AMT_ANNUITY'].fillna(df['AMT_ANNUITY'].median())
df['AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'].fillna(df['AMT_GOODS_PRICE'].mean())
df['CNT_FAM_MEMBERS'] = df['CNT_FAM_MEMBERS'].fillna(df['CNT_FAM_MEMBERS'].mode()[0])
df['EXT_SOURCE_1'] = df['EXT_SOURCE_1'].fillna(df['EXT_SOURCE_1'].mean())
df['EXT_SOURCE_2'] = df['EXT_SOURCE_2'].fillna(df['EXT_SOURCE_2'].mean())
df['EXT_SOURCE_3'] = df['EXT_SOURCE_3'].fillna(df['EXT_SOURCE_3'].mean())
df['AMT_REQ_CREDIT_BUREAU_HOUR'] = df['AMT_REQ_CREDIT_BUREAU_HOUR'].fillna(df['AMT_REQ_CREDIT_BUREAU_HOUR'].mean())
df['AMT_REQ_CREDIT_BUREAU_DAY'] = df['AMT_REQ_CREDIT_BUREAU_DAY'].fillna(df['AMT_REQ_CREDIT_BUREAU_DAY'].mean())
df['AMT_REQ_CREDIT_BUREAU_WEEK'] = df['AMT_REQ_CREDIT_BUREAU_WEEK'].fillna(df['AMT_REQ_CREDIT_BUREAU_WEEK'].mean())
df['AMT_REQ_CREDIT_BUREAU_MON'] = df['AMT_REQ_CREDIT_BUREAU_MON'].fillna(df['AMT_REQ_CREDIT_BUREAU_MON'].mean())
df['AMT_REQ_CREDIT_BUREAU_QRT'] = df['AMT_REQ_CREDIT_BUREAU_QRT'].fillna(df['AMT_REQ_CREDIT_BUREAU_QRT'].mean())
df['AMT_REQ_CREDIT_BUREAU_YEAR'] = df['AMT_REQ_CREDIT_BUREAU_YEAR'].fillna(df['AMT_REQ_CREDIT_BUREAU_YEAR'].mean())
df['OBS_30_CNT_SOCIAL_CIRCLE'] = df['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(df['OBS_30_CNT_SOCIAL_CIRCLE'].median())
df['OBS_60_CNT_SOCIAL_CIRCLE'] = df['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(df['OBS_60_CNT_SOCIAL_CIRCLE'].median())
df['DEF_60_CNT_SOCIAL_CIRCLE'] = df['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(df['DEF_60_CNT_SOCIAL_CIRCLE'].median())
df['DEF_30_CNT_SOCIAL_CIRCLE'] = df['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(df['DEF_30_CNT_SOCIAL_CIRCLE'].median())

In [6]:
for i in df.columns:
       if i[-4:-1] == "_AV":
              df[i] = df[i].fillna(df[i].mean())
       if i[-4:-1] == "MOD":
              df[i] = df[i].fillna(df[i].mode()[0])
       if i[-4:-1] == "MED":
              df[i] = df[i].fillna(df[i].median())

In [7]:
df['OWN_CAR_AGE'] = df['OWN_CAR_AGE'].fillna(0)
df['NAME_TYPE_SUITE'] = df['NAME_TYPE_SUITE'].fillna(df['NAME_TYPE_SUITE'].mode()[0])
df['OCCUPATION_TYPE'] = df['OCCUPATION_TYPE'].fillna("Unknown")
df['DAYS_LAST_PHONE_CHANGE'] = df['DAYS_LAST_PHONE_CHANGE'].fillna(df['DAYS_LAST_PHONE_CHANGE'].mode()[0])

In [8]:
df = df.drop(columns=['SK_ID_CURR'])

In [9]:
for i in df.columns:
    if not pd.api.types.is_numeric_dtype(df[i]):  # Check for non-numeric type
        df[i] = LabelEncoder().fit_transform(df[i])

In [10]:
X = df.drop(columns=['TARGET'])
y = df['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
lr = LogisticRegression(max_iter=100000, n_jobs=-1, class_weight={1:3, 0:1})

In [61]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     25491
           1       0.00      0.00      0.00      2185

    accuracy                           0.92     27676
   macro avg       0.46      0.50      0.48     27676
weighted avg       0.85      0.92      0.88     27676

[[25490     1]
 [ 2185     0]]


In [11]:
ros = RandomOverSampler(random_state=42)  # Or other oversampling methods
X_rostrain, y_rostrain = ros.fit_resample(X_train, y_train)

In [63]:
lr.fit(X_rostrain, y_rostrain)
y_rospred = lr.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     25491
           1       0.00      0.00      0.00      2185

    accuracy                           0.92     27676
   macro avg       0.46      0.50      0.48     27676
weighted avg       0.85      0.92      0.88     27676

[[25490     1]
 [ 2185     0]]


In [12]:
rus = RandomUnderSampler(random_state=42)  # Or other undersampling methods
X_rustrain, y_rustrain = rus.fit_resample(X_train, y_train)

In [65]:
lr.fit(X_rustrain, y_rustrain)
y_ruspred = lr.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     25491
           1       0.00      0.00      0.00      2185

    accuracy                           0.92     27676
   macro avg       0.46      0.50      0.48     27676
weighted avg       0.85      0.92      0.88     27676

[[25490     1]
 [ 2185     0]]


In [66]:
dtn = DecisionTreeClassifier()
dtn.fit(X_train, y_train)
y_pred = dtn.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92     25491
           1       0.15      0.18      0.16      2185

    accuracy                           0.86     27676
   macro avg       0.54      0.55      0.54     27676
weighted avg       0.87      0.86      0.86     27676

[[23307  2184]
 [ 1794   391]]


In [67]:
dto = DecisionTreeClassifier()
dto.fit(X_rostrain, y_rostrain)
y_dtrospred = dto.predict(X_val)
print(classification_report(y_val, y_dtrospred))
print(confusion_matrix(y_val, y_dtrospred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92     25491
           1       0.15      0.16      0.16      2185

    accuracy                           0.86     27676
   macro avg       0.54      0.54      0.54     27676
weighted avg       0.87      0.86      0.86     27676

[[23471  2020]
 [ 1826   359]]


In [68]:
dtu = DecisionTreeClassifier()
dtu.fit(X_rustrain, y_rustrain)
y_pred = dtu.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.58      0.72     25491
           1       0.11      0.59      0.18      2185

    accuracy                           0.58     27676
   macro avg       0.53      0.59      0.45     27676
weighted avg       0.88      0.58      0.68     27676

[[14827 10664]
 [  895  1290]]


In [13]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(X_rustrain, y_rustrain)
y_pred = nb.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.55      0.69     25491
           1       0.10      0.57      0.17      2185

    accuracy                           0.55     27676
   macro avg       0.52      0.56      0.43     27676
weighted avg       0.87      0.55      0.65     27676

[[14072 11419]
 [  946  1239]]


In [70]:
# svm = SVC()
# svm.fit(X_rustrain, y_rustrain)
# y_pred = svm.predict(X_val)
# print(classification_report(y_val, y_pred))
# print(confusion_matrix(y_val, y_pred))